<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/nb_templates/circuit_expms_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

This will save files to your local machine if `save_files` is set to True.

# Change Inputs Here

In [1]:
task = "numwords"  # choose: numerals, numwords, months
prompt_types = ['done', 'lost', 'names']
num_samps_per_ptype = 512 #768 512

model_name = "gpt2-small"

task = "numwords"

heads_not_ablate = [(0, 1), (1, 5), (4, 4), (4, 10), (5, 8), (6, 1), (6, 6), (6, 10), (7, 2), (7, 6), (7, 11), (8, 1), (8, 6), (8, 8), (8, 9), (8, 11), (9, 1), (9, 5), (9, 7)]
mlps_not_ablate = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

save_files = True
load_graph_files = False
run_on_other_tasks = True

# Setup

In [2]:
%%capture
%pip install git+https://github.com/neelnanda-io/TransformerLens.git

In [3]:
import torch as t
from torch import Tensor
from jaxtyping import Float
from typing import List, Optional, Callable, Tuple
from functools import partial

import pickle
from google.colab import files

from graphviz import Digraph, Source
from IPython.display import display

In [4]:
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, ActivationCache

t.set_grad_enabled(False)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Load Model

In [5]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [6]:
!git clone https://github.com/apartresearch/seqcont_circuits.git

Cloning into 'seqcont_circuits'...
remote: Enumerating objects: 698, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 698 (delta 80), reused 130 (delta 48), pack-reused 534
Receiving objects: 100% (698/698), 15.94 MiB | 11.06 MiB/s, done.
Resolving deltas: 100% (443/443), done.


In [7]:
%cd /content/seqcont_circuits/src/iter_node_pruning

from dataset import Dataset
from metrics import *
from head_ablation_fns import *
from mlp_ablation_fns import *
from node_ablation_fns import *
from loop_node_ablation_fns import *

/content/seqcont_circuits/src/iter_node_pruning


In [8]:
%cd /content/seqcont_circuits/src/iter_edge_pruning

from edge_pruning_fns import *

/content/seqcont_circuits/src/iter_edge_pruning


# Load datasets

In [9]:
prompts_list = []

for i in prompt_types:
    file_name = f'/content/seqcont_circuits/data/{task}/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:num_samps_per_ptype]

len(prompts_list)

Van done in one. Hat done in two. Ring done in three. Desk done in four. Sun done in
Oil lost in one. Apple lost in two. Tree lost in three. Snow lost in four. Apple lost in
Marcus born in one. Victoria born in two. George born in three. Brandon born in four. Jamie born in


1536

In [10]:
pos_dict = {}
for i in range(len(model.tokenizer.tokenize(prompts_list[0]['text']))):
    pos_dict['S'+str(i)] = i

In [11]:
dataset_1 = Dataset(prompts_list, pos_dict, model.tokenizer)

In [12]:
file_name = f'/content/seqcont_circuits/data/{task}/randDS_{task}.pkl'
with open(file_name, 'rb') as file:
    prompts_list_2 = pickle.load(file)

In [13]:
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer)

## Metrics

In [14]:
model.reset_hooks(including_permanent=True)
logits_original = model(dataset_1.toks)
orig_score = get_logit_diff(logits_original, dataset_1)

# model.reset_hooks(including_permanent=True)
# logits_corrup = model(dataset_2.toks)
# corrup_score = get_logit_diff(abc_logits_original, dataset_2)

In [15]:
def logit_diff_perc(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_logit_diff: float = orig_score,
    dataset_1: Dataset = dataset_1,
) -> float:
    patched_logit_diff = get_logit_diff(logits, dataset_1)
    return (patched_logit_diff / clean_logit_diff)

In [16]:
import gc

del(logits_original)
# del(logits_corrup)
t.cuda.empty_cache()
gc.collect()

73

# Run Iter Edge Pruning

## get circuit

Get the node ablated circuit without edges ablated.

In [17]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
abl_model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

new_logits = model(dataset_1.toks)
new_score = get_logit_diff(new_logits, dataset_1)
circ_score = (100 * new_score / orig_score).item()
print(f"(cand circuit / full) %: {circ_score:.4f}")
del(new_logits)

(cand circuit / full) %: 81.1096


## head to head

In [18]:
qkv_to_HH = {} # qkv to dict

for head_type in ["q", "k", "v"]:
    head_to_head_results = {}
    for head in heads_not_ablate:
        print(head_type, head)
        model.reset_hooks()
        model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

        result = circ_path_patch_head_to_heads(
            circuit = heads_not_ablate,
            receiver_heads = [head],
            receiver_input = head_type,
            model = model,
            patching_metric = logit_diff_perc,
            new_dataset = dataset_2,
            orig_dataset = dataset_1
        )
        head_to_head_results[head] = result
    qkv_to_HH[head_type] = head_to_head_results

q (0, 1)
q (1, 5)
q (4, 4)
q (4, 10)
q (5, 8)
q (6, 1)
q (6, 6)
q (6, 10)
q (7, 2)
q (7, 6)
q (7, 11)
q (8, 1)
q (8, 6)
q (8, 8)
q (8, 9)
q (8, 11)
q (9, 1)
q (9, 5)
q (9, 7)
k (0, 1)
k (1, 5)
k (4, 4)
k (4, 10)
k (5, 8)
k (6, 1)
k (6, 6)
k (6, 10)
k (7, 2)
k (7, 6)
k (7, 11)
k (8, 1)
k (8, 6)
k (8, 8)
k (8, 9)
k (8, 11)
k (9, 1)
k (9, 5)
k (9, 7)
v (0, 1)
v (1, 5)
v (4, 4)
v (4, 10)
v (5, 8)
v (6, 1)
v (6, 6)
v (6, 10)
v (7, 2)
v (7, 6)
v (7, 11)
v (8, 1)
v (8, 6)
v (8, 8)
v (8, 9)
v (8, 11)
v (9, 1)
v (9, 5)
v (9, 7)


In [19]:
head_to_head_adjList = {}
for head_type in ["q", "k", "v"]:
    for head in heads_not_ablate:
        result = qkv_to_HH[head_type][head]
        filtered_indices = (result < 0.8) & (result != 0.0)
        rows, cols = filtered_indices.nonzero(as_tuple=True)
        sender_nodes = list(zip(rows.tolist(), cols.tolist()))
        head_with_type = head + (head_type,)
        head_to_head_adjList[head_with_type] = sender_nodes

## mlp to mlp

In [20]:
mlp_to_mlp_results = {}

# for layer in range(11, 0, -1):
for layer in reversed(mlps_not_ablate):
    print(layer)
    model.reset_hooks()
    model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)
    result = circ_path_patch_MLPs_to_MLPs(
        mlp_circuit = mlps_not_ablate,
        receiver_layers = [layer],
        model = model,
        patching_metric = logit_diff_perc,
        new_dataset = dataset_2,
        orig_dataset = dataset_1
    )
    mlp_to_mlp_results[layer] = result

11
10
9
8
7
6
5
4
3
2
1
0


In [21]:
mlp_to_mlp_adjList = {}
for mlp in mlps_not_ablate:
    result = mlp_to_mlp_results[mlp]
    filtered_indices = (result < 0.80) & (result != 0.0)
    filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
    mlp_to_mlp_adjList[mlp] = filtered_indices.tolist()

## head to mlp

In [22]:
head_to_mlp_results = {}

# for layer in range(11, 0, -1):
for layer in reversed(mlps_not_ablate):
    print(layer)
    model.reset_hooks()
    model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)
    result = circ_path_patch_head_to_mlp(
        circuit = heads_not_ablate,
        receiver_layers = [layer],
        model = model,
        patching_metric = logit_diff_perc,
        new_dataset = dataset_2,
        orig_dataset = dataset_1
    )
    head_to_mlp_results[layer] = result

11
10
9
8
7
6
5
4
3
2
1
0


In [23]:
head_to_mlp_adjList = {}
for layer in mlps_not_ablate:
    result = head_to_mlp_results[layer]
    filtered_indices = (result < 0.8) & (result != 0.0)
    rows, cols = filtered_indices.nonzero(as_tuple=True)
    sender_nodes = list(zip(rows.tolist(), cols.tolist()))
    head_to_mlp_adjList[layer] = sender_nodes

## mlp to head

In [24]:
qkv_mlp_to_HH = {} # qkv to dict

for head_type in ["q", "k", "v"]:
    mlp_to_head_results = {}
    for head in heads_not_ablate:
        print(head_type, head)
        model.reset_hooks()
        model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

        result = circ_path_patch_mlp_to_head(
            mlp_circuit = mlps_not_ablate,
            receiver_heads = [head],
            receiver_input = head_type,
            model = model,
            patching_metric = logit_diff_perc,
            new_dataset = dataset_2,
            orig_dataset = dataset_1
        )
        mlp_to_head_results[head] = result
    qkv_mlp_to_HH[head_type] = mlp_to_head_results

q (0, 1)
q (1, 5)
q (4, 4)
q (4, 10)
q (5, 8)
q (6, 1)
q (6, 6)
q (6, 10)
q (7, 2)
q (7, 6)
q (7, 11)
q (8, 1)
q (8, 6)
q (8, 8)
q (8, 9)
q (8, 11)
q (9, 1)
q (9, 5)
q (9, 7)
k (0, 1)
k (1, 5)
k (4, 4)
k (4, 10)
k (5, 8)
k (6, 1)
k (6, 6)
k (6, 10)
k (7, 2)
k (7, 6)
k (7, 11)
k (8, 1)
k (8, 6)
k (8, 8)
k (8, 9)
k (8, 11)
k (9, 1)
k (9, 5)
k (9, 7)
v (0, 1)
v (1, 5)
v (4, 4)
v (4, 10)
v (5, 8)
v (6, 1)
v (6, 6)
v (6, 10)
v (7, 2)
v (7, 6)
v (7, 11)
v (8, 1)
v (8, 6)
v (8, 8)
v (8, 9)
v (8, 11)
v (9, 1)
v (9, 5)
v (9, 7)


In [25]:
mlp_to_head_adjList = {}
for head_type in ["q", "k", "v"]:
    for head in heads_not_ablate:
        result = qkv_mlp_to_HH[head_type][head]
        filtered_indices = (result < 0.8) & (result != 0.0)
        filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
        head_with_type = head + (head_type,)
        mlp_to_head_adjList[head_with_type] = filtered_indices.tolist()

# Save graph files to free up memory

In [26]:
if save_files:
    with open(task + "_head_to_head_results.pkl", "wb") as file:
        pickle.dump(head_to_head_results, file)
    files.download(task + "_head_to_head_results.pkl")

    with open(task + "_mlp_to_mlp_results.pkl", "wb") as file:
        pickle.dump(mlp_to_mlp_results, file)
    files.download(task + "_mlp_to_mlp_results.pkl")

    with open(task + "_head_to_mlp_results.pkl", "wb") as file:
        pickle.dump(head_to_mlp_results, file)
    files.download(task + "_head_to_mlp_results.pkl")

    with open(task + "_mlp_to_head_results.pkl", "wb") as file:
        pickle.dump(mlp_to_head_results, file)
    files.download(task + "_mlp_to_head_results.pkl")

del(head_to_head_results)
del(mlp_to_mlp_results)
del(head_to_mlp_results)
del(mlp_to_head_results)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run Iter Edge Pruning- resid post

## head to resid

In [27]:
model.reset_hooks()
model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

path_patch_head_to_final_resid_post = get_path_patch_head_to_final_resid_post(heads_not_ablate, model, logit_diff_perc,
                                                                new_dataset = dataset_2, orig_dataset = dataset_1)

In [28]:
heads_to_resid = {}
result = path_patch_head_to_final_resid_post
filtered_indices = (result < 0.8) & (result != 0.0)
rows, cols = filtered_indices.nonzero(as_tuple=True)
heads_to_resid['resid'] = list(zip(rows.tolist(), cols.tolist()))

## mlp to resid

In [29]:
model.reset_hooks()
model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

path_patch_mlp_to_final_resid_post = get_path_patch_mlp_to_final_resid_post(mlps_not_ablate, model, logit_diff_perc,
                                                                new_dataset = dataset_2, orig_dataset = dataset_1)

In [30]:
mlps_to_resid = {}
result = path_patch_mlp_to_final_resid_post
filtered_indices = (result < 0.8) & (result != 0.0)
filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
mlps_to_resid['resid'] = filtered_indices.tolist()

# Filter out nodes with no ingoing edges

In [31]:
head_to_head_adjList = {node: neighbors for node, neighbors in head_to_head_adjList.items() if neighbors}

In [32]:
mlp_to_head_adjList = {node: neighbors for node, neighbors in mlp_to_head_adjList.items() if neighbors}

# Save rest of graph files

In [33]:
if save_files:
    # graphs
    with open(task + "_head_to_head_adjList.pkl", "wb") as file:
        pickle.dump(head_to_head_adjList, file)
    files.download(task + "_head_to_head_adjList.pkl")

    with open(task + "_mlp_to_mlp_adjList.pkl", "wb") as file:
        pickle.dump(mlp_to_mlp_adjList, file)
    files.download(task + "_mlp_to_mlp_adjList.pkl")

    with open(task + "_head_to_mlp_adjList.pkl", "wb") as file:
        pickle.dump(head_to_mlp_adjList, file)
    files.download(task + "_head_to_mlp_adjList.pkl")

    with open(task + "_mlp_to_head_adjList.pkl", "wb") as file:
        pickle.dump(mlp_to_head_adjList, file)
    files.download(task + "_mlp_to_head_adjList.pkl")

    with open(task + "_heads_to_resid.pkl", "wb") as file:
        pickle.dump(heads_to_resid, file)
    files.download(task + "_heads_to_resid.pkl")

    with open(task + "_mlps_to_resid.pkl", "wb") as file:
        pickle.dump(mlps_to_resid, file)
    files.download(task + "_mlps_to_resid.pkl")

    # score results
    with open(task + "_heads_to_resid_results.pkl", "wb") as file:
        pickle.dump(path_patch_head_to_final_resid_post, file)
    files.download(task + "_heads_to_resid_results.pkl")

    with open(task + "_mlps_to_resid_results.pkl", "wb") as file:
        pickle.dump(path_patch_mlp_to_final_resid_post, file)
    files.download(task + "_mlps_to_resid_results.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Circuit graph plot

## load data

In [34]:
if load_graph_files:
    with open(task + "_head_to_head_adjList.pkl", "rb") as file:
        head_to_head_adjList = pickle.load(file)

    with open(task + "_mlp_to_mlp_adjList.pkl", "rb") as file:
        mlp_to_mlp_adjList = pickle.load(file)

    with open(task + "_head_to_mlp_adjList.pkl", "rb") as file:
        head_to_mlp_adjList = pickle.load(file)

    with open(task + "_mlp_to_head_adjList.pkl", "rb") as file:
        mlp_to_head_adjList = pickle.load(file)

    with open(task + "_heads_to_resid.pkl", "rb") as file:
        heads_to_resid = pickle.load(file)

    with open(task + "_mlps_to_resid.pkl", "rb") as file:
        mlps_to_resid = pickle.load(file)

## plot qkv

In [35]:
def plot_graph_adjacency_qkv(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                             mlp_to_head_adjList, heads_to_resid, mlps_to_resid,
                             filename="circuit_graph", highlighted_nodes=None):
    dot = Digraph()
    dot.attr(ranksep='0.45', nodesep='0.11')  # vert height- ranksep, nodesep- w

    dot.node('resid_post', color="#ffcccb", style='filled')

    for node in mlp_to_mlp_adjList.keys():
        sender_name = "MLP " + str(node)
        dot.node(sender_name, color="#ffcccb", style='filled')

    for node in head_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        dot.node(sender_name, color="#ffcccb", style='filled')
        sender_name = f"{node[0]} , {node[1]}"
        dot.node(sender_name, color="#ffcccb", style='filled')

    edges_added = []
    # for every q k v node, plot an edge to output node
    for node in head_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        receiver_name = f"{node[0]} , {node[1]}"
        dot.edge(sender_name, receiver_name, color = 'red')
        edges_added.append((sender_name, receiver_name))

    for node in mlp_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        dot.node(sender_name, color="#ffcccb", style='filled')
        sender_name = f"{node[0]} , {node[1]}"
        dot.node(sender_name, color="#ffcccb", style='filled')

    # for every q k v node, plot an edge to output node
    for node in mlp_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        receiver_name = f"{node[0]} , {node[1]}"
        if (sender_name, receiver_name) not in edges_added:
            dot.edge(sender_name, receiver_name, color = 'red')

    def loop_adjList(adjList):
        for end_node, start_nodes_list in adjList.items():
            if isinstance(end_node, int):
                receiver_name = "MLP " + str(end_node)
            elif isinstance(end_node, tuple):
                if len(end_node) == 3:
                    receiver_name = f"{end_node[0]} , {end_node[1]} {end_node[2]}"
                elif len(end_node) == 2:
                    receiver_name = f"{end_node[0]} , {end_node[1]}"
            else:
                receiver_name = 'resid_post'
            for start in start_nodes_list:
                if isinstance(start, int):
                    sender_name = "MLP " + str(start)
                elif isinstance(start, tuple):
                    if len(start) == 3:
                        sender_name = f"{start[0]} , {start[1]} {start[2]}"
                    elif len(start) == 2:
                        sender_name = f"{start[0]} , {start[1]}"
                dot.node(sender_name, color="#ffcccb", style='filled')
                dot.node(receiver_name, color="#ffcccb", style='filled')
                dot.edge(sender_name, receiver_name, color = 'red')

    loop_adjList(head_to_head_adjList)
    loop_adjList(mlp_to_mlp_adjList)
    loop_adjList(head_to_mlp_adjList)
    loop_adjList(mlp_to_head_adjList)
    loop_adjList(heads_to_resid)
    loop_adjList(mlps_to_resid)

    # Display the graph in Colab
    # display(Source(dot.source))

    # Save the graph to a file
    dot.format = 'png'  # You can change this to 'pdf', 'png', etc. based on your needs
    dot.render(filename)
    files.download(filename + ".png")

In [36]:
plot_graph_adjacency_qkv(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                         mlp_to_head_adjList, heads_to_resid, mlps_to_resid, filename="qkv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## plot no qkv fn

In [37]:
def plot_graph_adjacency(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                             mlp_to_head_adjList, heads_to_resid, mlps_to_resid,
                             filename="circuit_graph", highlighted_nodes=None):
    dot = Digraph()
    dot.attr(ranksep='0.45', nodesep='0.11')  # vert height- ranksep, nodesep- w

    def loop_adjList(adjList):
        for end_node, start_nodes_list in adjList.items():
            if isinstance(end_node, int):
                receiver_name = "MLP " + str(end_node)
            elif isinstance(end_node, tuple):
                receiver_name = f"{end_node[0]} , {end_node[1]}"
            else:
                receiver_name = 'resid_post'
            for start in start_nodes_list:
                if isinstance(start, int):
                    sender_name = "MLP " + str(start)
                elif isinstance(start, tuple):
                    sender_name = f"{start[0]} , {start[1]}"
                dot.node(sender_name, color="#ffcccb", style='filled')
                dot.node(receiver_name, color="#ffcccb", style='filled')
                dot.edge(sender_name, receiver_name, color = 'red')

    loop_adjList(head_to_head_adjList)
    loop_adjList(mlp_to_mlp_adjList)
    loop_adjList(head_to_mlp_adjList)
    loop_adjList(mlp_to_head_adjList)
    loop_adjList(heads_to_resid)
    loop_adjList(mlps_to_resid)

    # Save the graph to a file
    dot.format = 'png'  # You can change this to 'pdf', 'png', etc. based on your needs
    dot.render(filename)
    files.download(filename + ".png")

In [38]:
plot_graph_adjacency(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                         mlp_to_head_adjList, heads_to_resid, mlps_to_resid, filename="no qkv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>